# Embeddings for joint distribution learning

In [1]:
import numpy as np
import mosek
from mosek.fusion import *
%load_ext autoreload
%autoreload 2

In [2]:
from numpy import array, dot
from numpy import linalg as LA

In [3]:
import scipy as scip
from scipy.spatial.distance import pdist, squareform
from scipy.stats import norm

In [4]:
import itertools
from sklearn.metrics.pairwise import pairwise_kernels

### Algorithms used in the embeddings

#### Gaussian Kernel

In [5]:
def Gaussian_kernel_matrix(v, sig_v):
    pairwise_dists = squareform(pdist(v, 'euclidean'))
    K = np.exp(-np.power(pairwise_dists,2) / sig_v ** 2)
    return K

In [6]:
def Kernel_Vector_Double(z_s,z_k,sig):
    first_column = z_s[:,0]
    second_column = z_s[:,1]
    j_1 = np.exp(-np.power(first_column-z_k[0],2)/ sig[0])
    j_2 = np.exp(-np.power(second_column-z_k[1],2)/ sig[1])
    return j_1 * j_2

In [7]:
def Kernel_Vector_Single(y_s,y_k,sig_y):
    return np.exp(-np.power(y_s-y_k,2)/ sig_y)

#### Biorthogonal chokesky basis

In [8]:
def biorthogonl_chokesky_basis(K, epsilon):
    N = np.shape(K)[0]
    m = 1
    d = np.diag(K)
    #print(np.shape(d))   
    L = np.ones(N)
    B = np.ones(N)   
    j_s = []
    err = np.linalg.norm(d)
    j = np.argmax(d)
    j_s.append(j)
    l_m = K[:,j] 
    b_m = np.eye(1,N, j).flatten().T
    l_m = l_m / np.sqrt(d[j])
    b_m = b_m / np.sqrt(d[j])
    #print(np.shape(b_m))
    L =  np.c_[L,l_m]
    B = np.c_[B,b_m]
    L = L[:,1:]
    B = B[:,1:]
    d = d - np.multiply(l_m,l_m)
    err = np.linalg.norm(d,1)
    m = m + 1
    while err > epsilon:
        j = np.argmax(d)
        #print('j is : ',j)
        j_s.append(j)
        l_j = np.dot(L,L.T[:,j])
        b_j = np.dot(B,L.T[:,j])
        l_m = K[:,j] - l_j
        b_m = np.eye(1,N,j).flatten().T - b_j
        l_m = l_m / np.sqrt(d[j])
        b_m = b_m / np.sqrt(d[j])
        L = np.c_[L,l_m]
        B = np.c_[B,b_m]
        #print('L shape is :',np.shape(L))
        #print('B shape is :',np.shape(B))
        d = d - np.multiply(l_m,l_m)
        err = np.linalg.norm(d,1)
        m = m + 1
        #print('m is : ',m)
        #print('err is',err)
    return L, B, j_s

### Definition of the low rank embedding variables

#### Biorthogonal basis and normalization constraints

In [9]:
def biorthogonal_basis_decompostion_and_normalization_constraints(x, y, epsilon, sig_x, sig_y):
    
    
## Biorthogonal basis for the tensor product

    n = np.shape(x)[0]
    N = np.power(n,2)
    
    K_X = Gaussian_kernel_matrix([[i] for i in x], sig_x)
    K_Y = Gaussian_kernel_matrix([[i] for i in y], sig_y)

    L_X, B_X, j_s_X = biorthogonl_chokesky_basis(K_X,epsilon)
    L_Y, B_Y, j_s_Y = biorthogonl_chokesky_basis(K_Y,epsilon)

    m_X = np.shape(B_X)[1]
    m_Y = np.shape(B_Y)[1]
    m = m_X * m_Y

#Eignevalue decomposition of L.T L
    matrix_to_decompose_X = np.dot(L_X.T,L_X)

    w_X, v_X = LA.eig(matrix_to_decompose_X)
    U_X = B_X[j_s_X,:]
    Q_X = np.dot(U_X,v_X)

    matrix_to_decompose_Y = np.dot(L_Y.T,L_Y)

    w_Y, v_Y = LA.eig(matrix_to_decompose_Y)
    U_Y = B_Y[j_s_Y,:]
    Q_Y = np.dot(U_Y,v_Y)

    Q = np.kron(Q_X,Q_Y)
    Lambda = np.kron(w_X,w_Y)
    z_j_s = np.asarray(list(itertools.product(x[j_s_X], y[j_s_Y])))
    
# Constraints
    ## Normalization Constraints
    y_j_s = y[j_s_Y]
  
    matrix_Y = list(map(lambda y_sample: np.dot(Kernel_Vector_Single(y_j_s,y_sample,sig_y),Q_Y), y))
    vec_Y = np.sum(matrix_Y, axis = 0)
    I_X_Psi_Y = (1/n) * vec_Y
    
    x_j_s = x[j_s_X]
    matrix_X = list(map(lambda y_sample: np.dot(Kernel_Vector_Single(x_j_s,y_sample,sig_x),Q_X), x))
    vec_X = np.sum(matrix_X, axis = 0)
    I_Y_Psi_X = (1/n) * vec_X
    
    Unit_m_X = np.ones(m_X)
    Unit_m_Y = np.ones(m_Y)

    gamma_x = list(itertools.product(Unit_m_X,I_X_Psi_Y))
    Gamma_X = [a_tuple[0] * a_tuple[1] for a_tuple in gamma_x]

    gamma_y = list(itertools.product(I_Y_Psi_X,Unit_m_Y))
    Gamma_Y = [a_tuple[0] * a_tuple[1] for a_tuple in gamma_y]

    ## Q is the matrix that yields the biorthogonal chokesky basis, indeed the basis is given by \phi_p * Q
    ## Moreover, \phi_p is given by z, indeed \phi_p(z) is given by Kernel_Vector(z,z,sig)
    ## Thus \psi(z) = np.dot(Kernel_Vector(z,z,sig),Q)
    return Q, Lambda, z_j_s, m, Gamma_X, Gamma_Y

In [10]:
def positivity_constraint(m):
    a = np.zeros(m)
    b = np.ones(m)
    return a,b

In [11]:
def objective_function_inner_products(z_j_s, z, z_itertools, sig):

    n_tr = np.shape(z)[0]
    N = np.power(n_tr,2)
    matrix_1 = list(map(lambda z_i: Kernel_Vector_Double(z_j_s,z_i,sig),z))
    vec_1 = np.sum(matrix_1, axis = 0)

    matrix_2 = list(map(lambda z_ij: Kernel_Vector_Double(z_j_s,z_ij,sig),z_itertools))
    vec_2 = np.sum(matrix_2, axis = 0)
    
    return  (1/n_tr) * vec_1, (1/N) * vec_2

def objective_function_first_term(z_j_s, z, z_itertools, sig, Q):
    first_inner_product, second_inner_product = objective_function_inner_products(z_j_s, z, z_itertools, sig) 
    vector =  first_inner_product - second_inner_product
    return np.dot(vector,Q)

### Defintion of the greedy emebdding variables

In [12]:
def biorthogonal_basis_decompostion(y, epsilon, sig_y):
    
## Biorthogonal basis for the tensor product

    n = np.shape(y)[0]
    N = np.power(n,2)
    
    K_Y = Gaussian_kernel_matrix([[i] for i in y], sig_y)

    L_Y, B_Y, j_s_Y = biorthogonl_chokesky_basis(K_Y,epsilon)

    m_Y = np.shape(B_Y)[1]

#Eignevalue decomposition of L.T L
    matrix_to_decompose_Y = np.dot(L_Y.T,L_Y)

    w_Y, v_Y = LA.eig(matrix_to_decompose_Y)
    U_Y = B_Y[j_s_Y,:]
    Q_Y = np.dot(U_Y,v_Y)

    y_j_s = y[j_s_Y]

    ## Q is the matrix that yields the biorthogonal chokesky basis, indeed the basis is given by \phi_p * Q
    ## Moreover, \phi_p is given by z, indeed \phi_p(z) is given by Kernel_Vector(z,z,sig)
    ## Thus \psi(z) = np.dot(Kernel_Vector(z,z,sig),Q)
    
    return Q_Y, w_Y, y_j_s, m_Y

In [13]:
def first_normalization_cosntraint(y, y_j_s, Q_Y, sig_y):
# Constraints
    ## Normalization Constraints
    n = np.shape(y)[0]
    matrix_Y = list(map(lambda y_sample: np.dot(Kernel_Vector_Single(y_j_s,y_sample,sig_y),Q_Y), y))
    vec_Y = np.sum(matrix_Y, axis = 0)
    I_X_Psi_Y = (1/n) * vec_Y
    
    ## Q is the matrix that yields the biorthogonal chokesky basis, indeed the basis is given by \phi_p * Q_Y
    ## Moreover, \phi_p is given by z, indeed \phi_p(y) is given by Kernel_Vector(y_j_s,y,sig_y)
    ## Thus \psi(y) = np.dot(Kernel_Vector(y_j_s,y,sig_y),Q_Y)
    
    return I_X_Psi_Y

In [15]:
def second_normalization_constraint(h_s, P_X_s, P_X_i):
    result = np.matmul(P_X_s, h_s)
    threshold =  (1 / P_X_i) * result
    rhs = (1/P_X_i) * (1 - np.sum(P_X_s) - P_X_i)
    return threshold, rhs

In [16]:
def greedy_objective_function_inner_products(y, y_j_s, y_i, n_tr, sig_y):

    n = np.shape(y_i)[0]
    N = np.power(n_tr,2)
    matrix_1 = list(map(lambda y_sample: Kernel_Vector_Single(y_j_s, y_sample,sig_y),y_i))
    vec_1 = np.sum(matrix_1, axis = 0)

    matrix_2 = list(map(lambda y_sample: Kernel_Vector_Single(y_j_s, y_sample,sig_y), y))
    vec_2 = np.sum(matrix_2, axis = 0)
    
    return  (1/n_tr) * vec_1, (n/N) * vec_2

def greedy_objective_function_first_term(y_j_s, y_i, y, n_tr, sig_y, Q_Y):
    first_inner_product, second_inner_product = greedy_objective_function_inner_products(y, y_j_s, y_i, n_tr, sig_y) 
    vector =  first_inner_product - second_inner_product
    return np.dot(vector,Q_Y)

## Embeddings

### Low rank embedding

In [9]:
def low_rank_embedding(z, sig, epsilon, lambda_):
    x = z[:,0]
    y = z[:,1]
    z_itertools = np.asarray(list(itertools.product(y,x))) 
    Q, Lambda, z_j_s, m, Gamma_X, Gamma_Y = biorthogonal_basis_decompostion_and_normalization_constraints(x,y,epsilon, sig[0], sig[1])
    a, b = positivity_constraint(m)
    A = 0.5 * (np.diag(Lambda) + lambda_ * np.identity(m)) 
    first_term = objective_function_first_term(z_j_s, z, z_itertools, sig, Q)
    sqrt_quad_form = np.sqrt(Lambda)
    Model.putlicensepath(r"mosek.lic")
    with Model('cq01') as M:
        ## Creating variables
        h = M.variable('h', m, Domain.unbounded())
        h_p = M.variable('h_p', m, Domain.greaterThan(0.0))
        h_n = M.variable('h_n', m, Domain.greaterThan(0.0))
        u = M.variable('u', Domain.greaterThan(0.0))
        ## Imposing constraints
        M.constraint("posnegc",Expr.sub(h,Expr.sub(h_p,h_n)), Domain.equalsTo(0.0))
        ###Positivity constraint
        M.constraint("positivity",Expr.sum(Expr.sub(Expr.mulElm(h_n,b),Expr.mulElm(h_p,a))), Domain.lessThan(1.0))
        ###Normalization constraints
        M.constraint("normalization1", Expr.dot(np.real(Gamma_X),h), Domain.equalsTo(0.0))
        M.constraint("normalization2", Expr.dot(np.real(Gamma_Y),h), Domain.equalsTo(0.0))
        ## Quadratic cone constraint
        M.constraint("qc1", Expr.vstack(0.5,u,Expr.mulElm(np.real(sqrt_quad_form),h)), Domain.inRotatedQCone())
        M.setLogHandler(sys.stdout)
        ##Objective function 
        M.objective("obj", ObjectiveSense.Minimize, Expr.add(u,Expr.mul(2.0,Expr.dot(h,   np.real(first_term)))))
        # Solve the problem
        M.solve()
        M.writeTask('cqo1.opf')
        h_constrained = h.level()
        print('h = %s' % str(h_constrained))
    return h_constrained, z_j_s, Q

### Traditional embedding

In [3]:
def traditional_embedding(x, lambda_, sig_x):
    n = np.shape(x)[0]
    K_X = Gaussian_kernel_matrix([[i] for i in x], sig_x)
    return np.linalg.inv(K_X + lambda_ * np.identity(n))    

### Greddy embedding

In [10]:
def greedy_embedding(z, sig_y, epsilon, lambda_):
    z = z[z[:, 0].argsort()]
    x = z[:,0]
    y = z[:,1]
    n = np.shape(y)[0]
    D = x[-1]
    
    unique, counts = np.unique(x, return_counts=True)
    P_X_s = counts / n
    
    Q_Y, w_Y, y_j_s, m_Y = biorthogonal_basis_decompostion(y, epsilon, sig_y)
    a, b = positivity_constraint(m_Y)
    I_X_Psi_Y = first_normalization_cosntraint(y, y_j_s, Q_Y, sig_y)
    h_s = [] 
    A = 0.5 * (np.diag(w_Y) + lambda_ * np.identity(m_Y)) 
    sqrt_quad_form = np.sqrt(w_Y)

    y_0 = y[x == unique[0]]
    first_term_0 = greedy_objective_function_first_term(y_j_s, y_0, y, n, sig_y, Q_Y)
    Model.putlicensepath(r"mosek.lic")
    with Model('cq02') as M:
        ## Creating variables
        h_0 = M.variable('h_0', m_Y, Domain.unbounded())
        h_0_p = M.variable('h_0_p', m_Y, Domain.greaterThan(0.0))
        h_0_n = M.variable('h_0_n', m_Y, Domain.greaterThan(0.0))
        u = M.variable('u', Domain.greaterThan(0.0))
        ## Imposing constraints
        M.constraint("posnegc",Expr.sub(h_0,Expr.sub(h_0_p,h_0_n)), Domain.equalsTo(0.0))
        ###Positivity constraint
        M.constraint("positivity",Expr.sum(Expr.sub(Expr.mulElm(h_0_n,b),Expr.mulElm(h_0_p,a))), Domain.lessThan(1.0))
        ###Normalization constraints
        M.constraint("normalization1", Expr.dot(np.real(I_X_Psi_Y),h_0), Domain.equalsTo(0.0))
        ## Quadratic cone constraint
        M.constraint("qc1", Expr.vstack(0.5,u,Expr.mulElm(np.real(sqrt_quad_form),h_0)), Domain.inRotatedQCone())
        M.setLogHandler(sys.stdout)
        ##Objective function 
        M.objective("obj", ObjectiveSense.Minimize, Expr.add(u,Expr.mul(2.0,Expr.dot(h_0, np.real(first_term_0)))))
        # Solve the problem
        M.solve()
        M.writeTask('cqo2.opf')
        solh = h_0.level()
        h_s.append(solh) 
    h_s = np.asarray(h_s)    
    for i in np.arange(1,int(len(unique))):
        y_i = y[x == unique[i]]
        first_term_i = greedy_objective_function_first_term(y_j_s, y_i, y, n, sig_y, Q_Y)
        with Model('cq01') as M:
            ## Local variables
            print(np.shape(h_s))
            print(np.shape(P_X_s[:int(i)]))
            threshold, rhs = second_normalization_constraint(h_s, P_X_s[:int(i)], P_X_s[int(i)])
            ## Creating variables
            h_i = M.variable('h_i', m_Y, Domain.unbounded())
            h_i_p = M.variable('h_i_p', m_Y, Domain.greaterThan(0.0))
            h_i_n = M.variable('h_i_n', m_Y, Domain.greaterThan(0.0))
            h_i_u = M.variable('h_i_u', m_Y, Domain.greaterThan(-threshold))
            h_i_l = M.variable('h_i_l', m_Y, Domain.greaterThan(-threshold))
            u = M.variable('u', Domain.greaterThan(0.0))
            ## Imposing constraints
            M.constraint("posnegc",Expr.sub(h_i,Expr.sub(h_i_p,h_i_n)), Domain.equalsTo(0.0))
            M.constraint("specialposnegc",Expr.sub(h_i,Expr.sub(h_i_u,h_i_l)), Domain.equalsTo(0.0))
            ###Positivity constraint
            M.constraint("positivity",Expr.sum(Expr.sub(Expr.mulElm(h_i_n,b),Expr.mulElm(h_i_p,a))), Domain.lessThan(1.0))
            ###Normalization constraints
            M.constraint("normalization1", Expr.dot(np.real(I_X_Psi_Y),h_i), Domain.equalsTo(0.0))
            M.constraint("normalization2", Expr.sum(Expr.sub(Expr.mulElm(Expr.add(h_i_u,threshold),b),Expr.mulElm(Expr.add(h_i_l,threshold),a))), Domain.lessThan(1.0))
            ## Quadratic cone constraint
            M.constraint("qc1", Expr.vstack(0.5,u,Expr.mulElm(sqrt_quad_form,h_i)), Domain.inRotatedQCone())
            M.setLogHandler(sys.stdout)
            ##Objective function 
            M.objective("obj", ObjectiveSense.Minimize, Expr.add(u,Expr.mul(2.0,Expr.dot(h_i, first_term_i))))
            # Solve the problem
            M.solve()
            M.writeTask('cqo1.opf')
            solh = h_i.level()
        print('h = %s' % str(solh))
        h_s = np.vstack((h_s, solh))
    h_s = np.array(h_s)
    return h_s, unique, y_j_s, Q_Y